### Hangul
### Example 2 Table Input

In [1]:
import win32com.client as win32
import win32gui
import shutil  # 파일복사용 모듈
import time
import os

#hwp = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")  # Word.application
# https://martinii.fun/246

hwp = win32.gencache.EnsureDispatch('HWPFrame.HwpObject')
hwp.RegisterModule("FilePathCheckDLL", "SecurityModule")  # 보안모듈

True

In [2]:
CSV_DIR = "C:\\Users\\oakyo\\Desktop\\보고서"
BASE_DIR = "C:\\Users\\oakyo\\Desktop\\보고서\\notes"
BASE_DIR_IMAGE = "C:\\Users\\oakyo\\Desktop\\보고서\\zoom-dump"
UNI_CSV = "{0}\\uni_csv-1024.csv".format(CSV_DIR)
IRO_CSV = "{0}\\iro_csv-1024.csv".format(CSV_DIR)

In [3]:
import pandas as pd
import numpy as np

In [4]:
UNI_DF = pd.read_csv(UNI_CSV)
IRO_DF = pd.read_csv(IRO_CSV)

In [5]:
# IRO_DF.columns

In [6]:
UNI_DF.columns

Index(['Unnamed: 0', '주제', 'ID', '시작 시간', '파일 크기', '파일 크기.1', '총 보기 수',
       '총 다운로드 수', '마지막으로 액세스됨', 'Unnamed: 8', 'Unnamed: 9', 'U', 'I',
       'students', 'notes', 'zoom', 'URL'],
      dtype='object')

In [5]:
%cd C:\Users\oakyo\Desktop\보고서\iro-report

C:\Users\oakyo\Desktop\보고서\iro-report


In [8]:
# %cd C:\Users\oakyo\Desktop\보고서\uni-report

C:\Users\oakyo\Desktop\보고서\uni-report


In [6]:
for index, row in IRO_DF.iterrows():
# for index, row in UNI_DF.iterrows():
    str_uni = row['시작 시간']
    print(index, str_uni)
    file_name = "IRO-{0}.hwp".format(str_uni)
    # file_name = "UNI-{0}.hwp".format(str_uni)
    file_name = file_name.replace(",", " ")
    file_name = file_name.replace(":", " ")
    shutil.copyfile(r".\..\report.hwp",  # 원본은 그대로 두고,
                r"{0}".format(file_name))  # 복사한 파일을 수정하려고 함.
    hwp.Open(r"C:\Users\oakyo\Desktop\보고서\iro-report\{0}".format(file_name),"HWP",None)
    # hwp.Open(r"C:\Users\oakyo\Desktop\보고서\uni-report\{0}".format(file_name),"HWP",None)
    hwp.GetFieldList("HWP",None)
    field_list = [i for i in hwp.GetFieldList("HWP",None).split("\x02")]  # 한/글 안의 누름틀 목록 불러오기
    
    # ['date', 'time', 'person', 'content0']
    
    DATE = row['시작 시간']
    HOURS = row['I']
    PERSON = row['students']
    CONTENTS = row['주제']
    URL = row['URL']
    NOTES = row['notes']
    ZOOM = row['zoom']
    
    time.sleep(1)
    hwp.PutFieldText('date', DATE)
    hwp.PutFieldText('time', HOURS)
    hwp.PutFieldText('person', PERSON)
    hwp.PutFieldText('students', PERSON)
    hwp.PutFieldText('content0', CONTENTS)
    hwp.PutFieldText('url', URL)
    time.sleep(1)
    
    hwp.HAction.GetDefault("TablePropertyDialog", hwp.HParameterSet.HShapeObject.HSet)
    hwp.HParameterSet.HShapeObject.ShapeTableCell.Editable = 1
    hwp.HParameterSet.HShapeObject.ShapeTableCell.CellCtrlData.name = f"note"
    hwp.HAction.Execute("TablePropertyDialog", hwp.HParameterSet.HShapeObject.HSet)
    
    hwp.HAction.Run("TableRightCell")
    hwp.HAction.Run("TableRightCell")
    hwp.HAction.Run("TableRightCell")
    hwp.HAction.Run("TableRightCell")
    
    hwp.HAction.GetDefault("TablePropertyDialog", hwp.HParameterSet.HShapeObject.HSet)
    hwp.HParameterSet.HShapeObject.ShapeTableCell.Editable = 1
    hwp.HParameterSet.HShapeObject.ShapeTableCell.CellCtrlData.name = f"zoom"
    hwp.HAction.Execute("TablePropertyDialog", hwp.HParameterSet.HShapeObject.HSet)
    
    hwp.MoveToField('note')  # 해당 필드로 이동
    hwp.InsertPicture(os.path.join(BASE_DIR, NOTES), Embedded=True,
                      sizeoption=3)  # 이미지 삽입
    
    time.sleep(1)
    
    hwp.MoveToField('zoom')  # 해당 필드로 이동
    hwp.InsertPicture(os.path.join(BASE_DIR_IMAGE, ZOOM), Embedded=True,
                      sizeoption=3)  # 이미지 삽입
    
    time.sleep(1)
    hwp.Save("HWP")
    time.sleep(1)

0 10월 20, 2022 16:28:01
1 10월 19, 2022 13:13:02
2 10월 17, 2022 17:53:03
3 10월 17, 2022 13:11:36
4 10월 06, 2022 13:05:25
5 10월 04, 2022 16:27:56
6 10월 04, 2022 12:50:13
7 9월 30, 2022 10:30
8 9월 20, 2022 13:20:55


TypeError: join() argument must be str, bytes, or os.PathLike object, not 'float'

In [ ]:
hwp.Quit()

In [ ]:
'''
page = 1
for field in field_list:  # 모든 누름틀에 각각,
        hwp.MoveToField(f'{field}{{{{{page}}}}}')  # 커서를 해당 누름틀로 이동(작성과정을 지켜보기 위함. 없어도 무관)
        hwp.PutFieldText(f'{field}{{{{{page}}}}}',  # f"{{{{{page}}}}}"는 "{{1}}"로 입력된다. {를 출력하려면 {{를 입력.
                         excel[field].iloc[page])  # hwp.PutFieldText("index{{1}}") 식으로 실행될 것.
    print(f'{page + 1}:{excel.name[page]}')  # 현재 입력이 진행되고 있는 한/글문서 페이지번호를 콘솔창에 출력
'''

In [ ]:
# hwp.MoveToField(j)  # 해당 필드로 이동
# hwp.InsertPicture(os.path.join(BASE_DIR, f"{content + j.split('_', maxsplit=1)[1]}.png"), Embedded=True,
#                   sizeoption=3)  # 이미지 삽입